##* 분별 레이블 (_bun) 달기
- 주식시장 개장시간(0901~1530) 동안의 분별 레이블만 존재
- 종목 분봉 데이터 기간: 202112290901~202401120947
- 종목 분봉 데이터를 뉴스 속보 기간에 맞춰서 자르기 : 202301010000 이후 (~202401120947)
- 뉴스 속보 기간: 20230101~20240216
##* 양쪽 테이블에 join을 위한 공통 컬럼 만들기
- tdate: Timestamp형식 문자열
- 종목 분봉데이터를 1분단위 데이터프레임 형식으로 통일(1440행/일)
- 분별 변동률과 레이블 NaN 값을 ffill로 처리
- 시간 순서상 뉴스가 나오고 나서의 그 영향으로 변동했을 것이라 가정하여 NaN이 아닌 다음 순간의 변동률과 레이블 값으로 대치
-- ex) 0901:H, 0902:null, 0903:null, 0904:U
  =>  0901:H, 0902:U   , 0903:U   , 0904:U
- left join으로 레이블 넣기

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
#!pip install JPype1
#!pip install konlpy
#!pip install pyMySQL
#!pip install -U finance-datareader

In [ ]:
#-- Import packages --
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
import sqlite3
from sqlalchemy import create_engine
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from datetime import datetime
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')
#-- Import packages :END --

In [ ]:
#-- get_flattened_labeled_sokbo()--------------------------------
# 레이블 작업 할 당일 속보 DB에서 읽어오기
def get_flattened_labeled_sokbo(ddate):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT * FROM {ddate}_sokbo_flatten_label"
  # DB 검색결과를 dataframe에 저장
  oneday_flattened_labeled_sokbo_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  #print(oneday_flattened_labeled_sokbo_df.head())
  # DB close
  conn.close()
  # df 반환
  return oneday_flattened_labeled_sokbo_df
#-- get_flattened_labeled_sokbo(): End ---------------------------

In [ ]:
# --get_one_stock_bun_label()-------------------------------------
# 종목 분별 레이블(bun_rate_label) 가져오기
def get_one_stock_bun_label(stock_code):
  # DB 연결 준비
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="패스워드", db="DB이름", charset = 'utf8')
  # 실행확인을 위한 화면 출력
  print('-*',stock_code)
  # DB에서 종목 데이터  추출위한 sql 준비
  sql = f"SELECT x.* FROM A{stock_code}_bun_rate_label x"
  # DB 검색결과를 dataframe에 저장
  stock_df = pd.read_sql_query(sql, conn)
  # DB close
  conn.close()
  # df 반환
  return stock_df
# --get_one_stock_bun_label() :END--------------------------------
# --set_bun_label_with_stock_code_null_process()------------------
# 비어있는 분별 레이블의 NUll 값을 ffill로 처리하면서 레이블 넣기
def set_bun_label_with_stock_code_null_process(stock_code,sdate,edate):
  # 한종목 분 레이블 읽어오기
  stock_df = get_one_stock_bun_label(stock_code)
  # stock_df를 20230101이후로 자르기, 종복 분봉데이타는 2년치, 뉴스는 1년치만 쓴다
  stock_df = stock_df[pd.to_datetime(stock_df['date'])>='20230101']

  #----------------------------------------------------
  #datetime형식의 날짜 컬럼 생성
  stock_df['tdate']= pd.to_datetime(stock_df['date'])
  # 최신이 처음으로 오도록 내림차순 정렬
  stock_df.sort_values('tdate', ascending=False, inplace=True, ignore_index=True)
  #실행확인을 위한 화면출력
  print(stock_df.head())
  # df 원본 복사
  temp_df = stock_df.copy()
  # 시작시간 설정 (현재)
  c_time= temp_df.iloc[0]['tdate']
  # 종료시간 설정(역순이므로 시작일 의미)
  e_time = '20230101'
  # 날짜 범위 생성
  date_range = pd.date_range(start=e_time, end=c_time, freq='1min')
  # left 조인을 위한 데이터프레임 생성
  left_temp_df = pd.DataFrame({'tdate': date_range})
  # 역순 정렬
  left_temp_df.sort_values('tdate', ascending=False, ignore_index=True, inplace=True)
  # left 조인, 날짜 기준으로 있는 분당 데이터는 들어오고, 1일 1440분당 행은 그대로, 값없는 것은 null
  lj_result_df = pd.merge(left_temp_df, temp_df, on='tdate', how='left')
  # ffill 옵션으로 bun_rate와 bun_label이 Nan 인 경우 앞 시간의 값 rate, label을 넣는다
  lj_result_df['bun_rate'] = lj_result_df['bun_rate'].fillna(method='ffill')
  lj_result_df['bun_label'] = lj_result_df['bun_label'].fillna(method='ffill')
  # 실행 확인을 위한 화면 출력
  print(lj_result_df.head(10))
  # 복사본 생성 대입
  stock_df=lj_result_df.copy()
#----------------------------------------------------
  # 컬럼명 수정: 종목번호 추가
  stock_df.rename(columns={'date':stock_code+'_date','bun_rate': stock_code+'_bun_rate','bun_label': stock_code+'_bun_label'}, inplace=True)

  #-- !! 날짜 바꾸려면 여기 수정하기!!---------------------------------
  # 뉴스 날짜 지정:시작 날짜
  #sdate = '20240111'
  # 뉴스 날짜 지정: 끝 날짜
  #edate = '20240112'
  #-----------------------------------------------------------

  # 처리 날짜 초기화
  ddate = sdate
  while pd.to_datetime(ddate)<=pd.to_datetime(edate):
    # 실행 확인을 위한 화면 출력
    print(ddate)
    # 해당 날짜 뉴스 테이블 읽어오기
    one_news_df= get_flattened_labeled_sokbo(ddate) #처음, 없을때
    #one_news_df = get_test_sokbo(ddate)
    # one_news_df에 jdate 만들기, 20230102 날짜 하루 뉴스
    #one_news_df['jdate']=pd.to_datetime(one_news_df['rdate']).dt.strftime('%Y%m%d%H%M')
    one_news_df['tdate']=pd.to_datetime(one_news_df['rdate'])#.dt.strftime('%Y%m%d%H%M')
    # 뉴스 속보 테이블에 종목 분별 레이블 join 시켜서 넣기(left join)
    left_join_result_df=pd.merge(one_news_df, stock_df, on='tdate', how='left')

    # DB 저장 연결 준비
    db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
    # DB 커넥션 생성
    db_connection = create_engine(db_connection_path)
    # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
    left_join_result_df.to_sql(name=ddate+'_sokbo_flatten_label', con=db_connection,if_exists='replace',index=False)
    # DB close
    db_connection.dispose()
    # 날짜를 하루 증가시키기
    ddate = pd.to_datetime(ddate)+pd.Timedelta(days=1)
    # YYYYMMDD 형식의 문자열로 변환
    ddate = ddate.strftime('%Y%m%d')


In [ ]:
#=========================================================================
# 메인 함수 호출: 예) 종목 '034950','20230101'~'20240112'
# 뉴스 속보에 분당 레이블 달기:
#-------------------------------------------------------------------------
set_bun_label_with_stock_code_null_process('034950','20230101','20240112')
#-------------------------------------------------------------------------

In [ ]:
#=========================================================================
# 메인 함수 호출: 예) 종목 '005930','20230101'~'20240112'
# 뉴스 속보에 분당 레이블 달기
#-------------------------------------------------------------------------
set_bun_label_with_stock_code_null_process('005930','20230101','20240112')
#=========================================================================

In [ ]:
#=========================================================================
# 메인 함수 호출: 예) 종목 '328380','20230101'~'20240112'
# 뉴스 속보에 분당 레이블 달기
#-------------------------------------------------------------------------
set_bun_label_with_stock_code_null_process('328380','20230101','20240112')
#-------------------------------------------------------------------------

In [ ]:
####################################################################################################

In [ ]:
# 날짜_sokbo_flatten_label 테이블에서 일별 레이블만 남기고 분별 레이블 결과 컬럼 지우기
def delete_bun_leave_only_il_at_sokbo(sdate,edate):
  ddate=sdate
  while pd.to_datetime(ddate)<=pd.to_datetime(edate):
    print(ddate)
    working_df = get_flattened_labeled_sokbo(ddate)
    tt_df=working_df[['rdate','news_cnt','2000_encoded_text_code','034950_il','005930_il','328380_il']]
    # DB 저장 연결 준비
    db_connection_path = 'mysql+pymysql://아이디:패스워드@호스트주소:포트번호/DB이름'
    # DB 커넥션 생성
    db_connection = create_engine(db_connection_path)
    # 데이터프레임을 DB로 저장, 이미 있으면 덮어쓰기(replace)
    tt_df.to_sql(name=ddate+'_sokbo_flatten_label', con=db_connection,if_exists='replace',index=False)
    # DB close
    db_connection.dispose()
    # 날짜를 하루 증가시키기
    ddate = pd.to_datetime(ddate)+pd.Timedelta(days=1)
    # YYYYMMDD 형식의 문자열로 변환
    ddate = ddate.strftime('%Y%m%d')


In [ ]:
delete_bun_leave_only_il_at_sokbo('20240113','20240216')

In [ ]:
#######################################################################################################